# Skyline chart

This notebook assembles the data needed to construct a regional skyline chart.

In [1]:
import numpy as np
import pandas as pd
import duckdb

In [2]:
region = 'eaeu'
input = 'adb-mrio.parquet'
year = 2022

outputpath = '../../Country Reports/Kazakhstan/data/interim/skyline.csv'

## Construct regional input–output table

In [3]:
G = 73
N = 35
f = 5

regions = pd.read_excel('../data/raw/countries.xlsx')
regions.dropna(subset=['mrio'], inplace=True)
regions.sort_values(by='mrio', inplace=True)
regions = regions[['mrio', f'rta_{region}']]
regions.fillna(0, inplace=True)
regions[f'rta_{region}'] = regions[f'rta_{region}'].astype(int)

### Load MRIO values

In [4]:
mrio = duckdb.sql(f"SELECT * EXCLUDE(t, si) FROM read_parquet('../data/mrio/{input}') WHERE t={year}").df()
mrio = mrio.values

x = mrio[-1][:(G*N)]
Z = pd.DataFrame(mrio[:(G*N)][:, :(G*N)])
Y_big = mrio[:(G*N)][:, (G*N):-1]
Y = Y_big @ np.kron(np.eye(G), np.ones((f, 1)))
Y = pd.DataFrame(Y)

### Calculate regional IOT values

In [5]:
# Output

df_x = pd.DataFrame({
    'region': regions[f'rta_{region}'].repeat(N),
    'i': np.tile(np.arange(1, N+1), G),
    'x': x
})
df_x = df_x.groupby(['region', 'i']).sum().reset_index()
x = df_x['x']

# Intermediates

df_Z = pd.DataFrame({
    'region': regions[f'rta_{region}'].repeat(N),
    'i': np.tile(np.arange(1, N+1), G)
}).reset_index(drop=True)
df_Z = pd.concat([df_Z, Z], axis=1)
df_Z = df_Z.groupby(['region', 'i']).sum().reset_index()

tZ = np.transpose(df_Z.iloc[:, 2:])
df_tZ = pd.DataFrame({
    'region': regions[f'rta_{region}'].repeat(N),
    'i': np.tile(np.arange(1, N+1), G)
}).reset_index(drop=True)
df_tZ = pd.concat([df_tZ, tZ], axis=1)
df_tZ = df_tZ.groupby(['region', 'i']).sum().reset_index()

Z = np.transpose(df_tZ.iloc[:, 2:])

# Final sales

df_Y = pd.DataFrame({
    'region': regions[f'rta_{region}'].repeat(N),
    'i': np.tile(np.arange(1, N+1), G)
}).reset_index(drop=True)
df_Y = pd.concat([df_Y, Y], axis=1)
df_Y = df_Y.groupby(['region', 'i']).sum().reset_index()

tY = np.transpose(df_Y.iloc[:, 2:])
df_tY = pd.DataFrame({'region': regions[f'rta_{region}']}).reset_index(drop=True)
df_tY = pd.concat([df_tY, tY], axis=1)
df_tY = df_tY.groupby(['region']).sum().reset_index()

Y = np.transpose(df_tY.iloc[:, 1:])

### Consolidate into table

In [6]:
Z_d = Z.iloc[:, N:N+N+1]
Y_d = Y[1]

riot = pd.DataFrame({'i': np.tile(np.arange(1, N+1), 2)})
riot = pd.concat([riot, pd.DataFrame(Z_d)], axis=1)
riot = pd.concat([riot, pd.DataFrame(Y_d)], axis=1)
riot = riot.groupby(['i']).sum().reset_index()

colnames = ['i'] + [f'Z_d_{i}' for i in range(1, 36)] + ['Y_d']
riot.columns = colnames

riot['Z_exports'] = np.sum(Z.iloc[N:N+N+1, 0:N], axis=1).reset_index(drop=True)
riot['Y_exports'] = Y.iloc[N:N+N+1, 0].reset_index(drop=True)
riot['imports'] = np.sum(riot.iloc[:, 1:], axis=1).reset_index(drop=True) - x[N:].reset_index(drop=True)
riot['x'] = x[N:].reset_index(drop=True)

In [7]:
riot

,i,Z_d_1,Z_d_2,Z_d_3,Z_d_4,Z_d_5,Z_d_6,Z_d_7,Z_d_8,Z_d_9,...,Z_d_31,Z_d_32,Z_d_33,Z_d_34,Z_d_35,Y_d,Z_exports,Y_exports,imports,x
0,1,37043.083830,713.567895,49488.836814,313.883450,50.464976,3375.915883,3478.089765,42.732285,297.649500,...,994.105229,247.946002,894.280523,251.590944,0.0,91981.429335,10266.675250,1183.684466,12023.701698,196699.002300
1,2,344.941003,10756.310281,612.782297,62.297499,9.455251,201.048641,187.489368,71941.535010,3833.463051,...,1439.420125,244.540346,288.897779,7685.444107,0.0,3520.687362,293333.909434,7074.177441,1405.073728,503419.525341
2,3,4994.307032,109.918586,21897.771366,27.430010,21.725466,7.787775,40.146786,36.413219,682.811352,...,1339.217223,612.745444,2257.024118,280.206526,0.0,147755.397693,8395.610401,7932.303883,17269.944738,184671.443242
3,4,249.748649,330.654483,182.801679,4245.883554,257.510340,24.340871,59.216280,15.327111,170.155144,...,430.617272,65.030548,353.103575,211.140551,0.0,35780.069144,317.815218,333.889615,32760.311517,15216.769179
4,5,7.053560,16.893949,6.495083,382.337931,203.830407,1.355248,4.872503,2.076955,7.719671,...,40.503936,7.251350,9.348192,13.785993,0.0,9019.933649,205.277812,296.460050,8205.182748,2407.926564
5,6,148.523703,378.922902,161.903035,10.068877,3.088543,1594.380171,220.339563,8.331668,157.610323,...,15.608444,21.716359,11.758275,91.475365,0.0,835.755936,7514.204445,18.983088,1569.862186,19285.838298
6,7,387.079773,595.251432,2478.990822,46.315784,12.267197,234.269059,5439.672890,170.460742,1685.389845,...,1344.880371,440.134468,449.887592,213.564611,0.0,6993.408132,4622.491536,153.758735,3686.974668,33372.308124
7,8,5290.171041,6485.354390,1361.590603,40.772851,10.874416,343.466153,515.471998,6475.073482,2720.604306,...,1506.226566,101.100315,466.917233,1661.603110,0.0,6549.033755,88398.615504,33599.762089,2153.302679,215460.066851
8,9,4782.047896,4772.610605,1924.223388,945.459785,148.729839,958.137966,1264.488237,3746.116266,26449.398038,...,859.988388,232.146069,3830.725909,869.723031,0.0,33732.199188,26011.599429,1306.748935,25213.984918,122124.142855
9,10,326.768278,3572.869417,512.034378,73.197015,24.992162,170.627200,227.774680,450.607448,847.778076,...,107.818810,122.467526,1603.960534,562.855791,0.0,7000.914490,2220.176976,106.800965,4164.224198,36395.775781


## Skyline chart

In [8]:
Z = riot.iloc[:, 1:N+1]
x = riot['x']
A = Z @ np.diag(np.where(x != 0, 1/x, 0))
B = np.linalg.inv(np.eye(N) - A)

sf = B @ riot['Y_d']
se = B @ (riot['Z_exports'] + riot['Y_exports'])
sm = B @ riot['imports']

df = pd.DataFrame({
    't': year,
    'i': np.arange(1, N+1),
    'x': x,
    'sf': sf, 
    'se': se, 
    'sm': sm, 
    'self_sufficiency': x/sf,
    'total': (sf+se)/sf,
    'imports': sm/sf
})

sectors = pd.read_excel('../data/raw/sectors.xlsx')
sectors = sectors.drop_duplicates(subset='ind', ignore_index=True)
sectors = sectors[['ind', 'abv', 'name']]
sectors.rename(columns={'ind':'i'}, inplace=True)

df = pd.merge(df, sectors)
df.insert(2, 'abv', df.pop('abv'))
df.insert(3, 'name', df.pop('name'))

df.to_csv(outputpath, index=False)

In [9]:
df

,t,i,abv,name,x,sf,se,sm,self_sufficiency,total,imports
0,2022,1,AHF,"Agriculture, hunting, forestry and fishing",196699.002300,196655.746588,27532.175834,27488.920122,1.000220,1.140002,0.139782
1,2022,2,MIN,Mining and quarrying,503419.525341,135421.605038,398367.189138,30369.268836,3.717424,3.941681,0.224257
2,2022,3,FBT,"Food, beverages and tobacco",184671.443242,185618.059598,20953.560437,21900.176792,0.994900,1.112885,0.117985
3,2022,4,TEX,Textiles and textile products,15216.769179,60752.963726,3490.421690,49026.616236,0.250470,1.057453,0.806983
4,2022,5,LTH,"Leather, leather products and footwear",2407.926564,12048.366528,791.849794,10432.289758,0.199855,1.065723,0.865868
5,2022,6,WDC,Wood and products of wood and cork,19285.838298,12633.750823,10154.761241,3502.673765,1.526533,1.803780,0.277247
6,2022,7,PPP,"Pulp, paper, printing and publishing",33372.308124,31615.446431,11225.122416,9468.260722,1.055570,1.355052,0.299482
7,2022,8,CRP,"Coke, refined petroleum and nuclear fuel",215460.066851,80639.813261,150877.884088,16057.630498,2.671882,2.871010,0.199128
8,2022,9,CCP,Chemicals and chemical products,122124.142855,117073.082699,57355.690146,52304.629989,1.043145,1.489914,0.446769
9,2022,10,RBP,Rubber and plastics,36395.775781,36250.137706,11001.220804,10855.582729,1.004018,1.303481,0.299463
